In [1]:
import os
from azureml.core import Workspace
from azureml.core import Environment
from azureml.core import ScriptRunConfig, Experiment
from azureml.widgets import RunDetails
from azureml.core import Model

In [2]:
# AMLワークスペースへの接続
ws = Workspace.from_config()

In [3]:
# training script
source_dir = "train"
script_name = "train_bert.py"

# environment file
environment_file = os.path.join("train", "train_bert_env.yml")

# azure ml settings
environment_name = "pl-env-lang"
experiment_name = "bert-livedoor"
compute_name = "shuit-gpu-ins02"

In [4]:
# 学習環境作成、初回のみ長時間
env = Environment.from_conda_specification(environment_name, environment_file)

env.docker.enabled = True
env.docker.base_image = (
    "mcr.microsoft.com/azureml/openmpi4.1.0-cuda11.0.3-cudnn8-ubuntu18.04"
)

'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [7]:
# 学習設定
src = ScriptRunConfig(
    source_directory=source_dir,
    script=script_name,
    arguments=["--batch_size", 256, "--max_epochs", 20, "--gpus", 1],
    compute_target=compute_name,
    environment=env,
)

In [8]:
# 実行
run = Experiment(ws, experiment_name).submit(src)
run.wait_for_completion(show_output=True)

RunId: bert-livedoor_1641985249_7c0fba4c
Web View: https://ml.azure.com/runs/bert-livedoor_1641985249_7c0fba4c?wsid=/subscriptions/902f236f-44df-463a-a5cb-1516ab2a9cd2/resourcegroups/shuit-common/workspaces/shuit-ml-workspace&tid=72f988bf-86f1-41af-91ab-2d7cd011db47

Streaming azureml-logs/55_azureml-execution-tvmps_a46938a6323b6ff5d91a38cb3c6b0b117014b72f62bcb10efd1e5b630f41cd0e_d.txt

2022-01-12T11:01:04Z Running following command: /bin/bash -c sudo blobfuse /mnt/batch/tasks/shared/LS_root/jobs/shuit-ml-workspace/azureml/bert-livedoor_1641985249_7c0fba4c/mounts/workspaceblobstore --tmp-path=/mnt/batch/tasks/shared/LS_root/jobs/shuit-ml-workspace/azureml/bert-livedoor_1641985249_7c0fba4c/caches/workspaceblobstore -o ro --file-cache-timeout-in-seconds=1000000 --cache-size-mb=619058 -o nonempty -o allow_other --config-file=/mnt/batch/tasks/shared/LS_root/jobs/shuit-ml-workspace/azureml/bert-livedoor_1641985249_7c0fba4c/configs/workspaceblobstore.cfg --log-level=LOG_WARNING
2022-01-12T11

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "User program failed with ModuleNotFoundError: No module named 'torchmetrics'",
        "messageParameters": {},
        "detailsUri": "https://aka.ms/azureml-run-troubleshooting",
        "details": []
    },
    "time": "0001-01-01T00:00:00.000Z"
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Activity Failed:\n{\n    \"error\": {\n        \"code\": \"UserError\",\n        \"message\": \"User program failed with ModuleNotFoundError: No module named 'torchmetrics'\",\n        \"messageParameters\": {},\n        \"detailsUri\": \"https://aka.ms/azureml-run-troubleshooting\",\n        \"details\": []\n    },\n    \"time\": \"0001-01-01T00:00:00.000Z\"\n}"
    }
}

In [66]:
# モデル登録
run.register_model(
    model_name="bert-livedoor-model",
    model_path=os.path.join('outputs', 'model.onnx'),
    model_framework=Model.Framework.ONNX,
)

Model(workspace=Workspace.create(name='shuit-ml-workspace', subscription_id='902f236f-44df-463a-a5cb-1516ab2a9cd2', resource_group='shuit-common'), name=bert-livedoor-model, id=bert-livedoor-model:4, version=4, tags={}, properties={})